In [34]:
import pandas as pd

In [35]:
df=pd.read_csv(r'C:\Users\bhask\OneDrive\Documents\KN-ML\Ineuron Materials\train.csv')
df2=pd.read_csv(r'C:\Users\bhask\OneDrive\Documents\KN-ML\Ineuron Materials\train.csv')

In [36]:
df.sample(1)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
115206,5H9RAB2Q,Male,42,RG254,Self_Employed,X2,37,No,1120269,Yes,0


In [37]:
df=df.drop(['ID'],axis=1)

In [38]:
df.isnull().sum()


Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         29325
Avg_Account_Balance        0
Is_Active                  0
Is_Lead                    0
dtype: int64

In [39]:
def impute_nan(df,i):
    df[i+"_random"]=df[i]
    random_sample=df[i].dropna().sample(df[i].isnull().sum(),random_state=0)
    random_sample.index=df[df[i].isnull()].index
    df.loc[df[i].isnull(),i+'_random']=random_sample
    df.drop([i],axis=1,inplace=True)

In [40]:
impute_nan(df,'Credit_Product')

In [41]:
df.isnull().sum()


Gender                   0
Age                      0
Region_Code              0
Occupation               0
Channel_Code             0
Vintage                  0
Avg_Account_Balance      0
Is_Active                0
Is_Lead                  0
Credit_Product_random    0
dtype: int64

In [42]:
b=list(df2['Region_Code'].unique())
print(len(b))

35


In [43]:
df_age=df[['Gender','Credit_Product_random','Is_Active']]

In [44]:
df_age = pd.get_dummies(df_age,drop_first=True)

In [45]:

df.drop(['Gender'],axis=1,inplace=True)
df.drop(['Credit_Product_random'],axis=1,inplace=True)
df.drop(['Is_Active'],axis=1,inplace=True)

In [46]:

df=pd.concat([df,df_age],axis=1)

In [47]:
df.sample(1)

,Age,Region_Code,Occupation,Channel_Code,Vintage,Avg_Account_Balance,Is_Lead,Gender_Male,Credit_Product_random_Yes,Is_Active_Yes
96238,77,RG283,Other,X3,51,602343,1,0,1,0


In [48]:
def ordinal(df,i,j):
    ordinal_labels=df.groupby([i])[j].mean().sort_values().index
    enumerate(ordinal_labels,0)
    ordinal_labels2={k:i for i,k in enumerate(ordinal_labels,0)}
    ordinal_labels2
    df[i+'_new']=df[i].map(ordinal_labels2)
    df.drop(i,inplace=True,axis=1)

In [49]:
ordinal(df,'Region_Code','Is_Lead')

In [50]:
df.sample(1)

,Age,Occupation,Channel_Code,Vintage,Avg_Account_Balance,Is_Lead,Gender_Male,Credit_Product_random_Yes,Is_Active_Yes,Region_Code_new
93184,55,Self_Employed,X2,21,4187770,1,0,1,0,16


In [51]:
ordinal(df,'Occupation','Is_Lead')

In [52]:
df.sample(1)

,Age,Channel_Code,Vintage,Avg_Account_Balance,Is_Lead,Gender_Male,Credit_Product_random_Yes,Is_Active_Yes,Region_Code_new,Occupation_new
105594,28,X1,19,191287,0,0,0,1,4,0


In [53]:
df.groupby(['Channel_Code'])['Is_Lead'].mean()

Channel_Code
X1    0.091421
X2    0.327895
X3    0.367025
X4    0.247800
Name: Is_Lead, dtype: float64

In [54]:
ordinal(df,'Channel_Code','Is_Lead')

In [55]:
df.sample(2)

,Age,Vintage,Avg_Account_Balance,Is_Lead,Gender_Male,Credit_Product_random_Yes,Is_Active_Yes,Region_Code_new,Occupation_new,Channel_Code_new
177578,34,15,858227,1,0,0,0,16,0,0
201099,57,68,555858,0,1,0,0,24,2,2


In [56]:
df.shape

(245725, 10)

In [57]:
x=df.drop(labels='Is_Lead', axis=1)
y= df['Is_Lead']

In [58]:
x.head()

,Age,Vintage,Avg_Account_Balance,Gender_Male,Credit_Product_random_Yes,Is_Active_Yes,Region_Code_new,Occupation_new,Channel_Code_new
0,73,43,1045696,0,0,0,32,1,3
1,30,32,581988,0,0,0,24,0,0
2,56,26,1484315,0,0,1,32,2,3
3,34,19,470454,1,0,0,4,0,0
4,30,33,886787,0,0,0,14,0,0


In [59]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_data=scaler.fit_transform(x)

c:\users\bhask\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\bhask\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [60]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(scaled_data,y,test_size=0.3,random_state=42)

In [61]:
# fit model to training data
from xgboost import XGBClassifier
model = XGBClassifier(objective='binary:logistic')
model.fit(train_x, train_y,verbose=True,early_stopping_rounds=10,eval_metric='aucpr',eval_set=[(test_x,test_y)])

c:\users\bhask\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-aucpr:0.56442
[1]	validation_0-aucpr:0.57051
[2]	validation_0-aucpr:0.57303
[3]	validation_0-aucpr:0.57534
[4]	validation_0-aucpr:0.57822
[5]	validation_0-aucpr:0.58230
[6]	validation_0-aucpr:0.58275
[7]	validation_0-aucpr:0.58366
[8]	validation_0-aucpr:0.58454
[9]	validation_0-aucpr:0.58500
[10]	validation_0-aucpr:0.58597
[11]	validation_0-aucpr:0.58665
[12]	validation_0-aucpr:0.58695
[13]	validation_0-aucpr:0.58743
[14]	validation_0-aucpr:0.58732
[15]	validation_0-aucpr:0.58764
[16]	validation_0-aucpr:0.58765
[17]	validation_0-aucpr:0.58736
[18]	validation_0-aucpr:0.58731
[19]	validation_0-aucpr:0.58757
[20]	validation_0-aucpr:0.58794
[21]	validation_0-aucpr:0.58791
[22]	validation_0-aucpr:0.58800
[23]	validation_0-aucpr:0.58800
[24]	validation_0-aucpr:0.58799
[25]	validation_0-aucpr:0.58791
[26]	validation_0-aucpr:0.58811
[27]	validation_0-aucpr:0.58795
[28]	validation_0-aucpr:0.58785
[29]	validation_0-aucpr:0.58769
[30]	validation_0-aucpr:0.58755
[31]	validation_0-

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=4, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       use_label_encoder=True, validate_parameters=1, verbosity=None)

In [62]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(train_x)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(train_y,predictions)
accuracy

0.8036475259727802

In [63]:
y_pred = model.predict(test_x)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(test_y,predictions)
accuracy

0.7979055318918039

In [64]:
from sklearn.model_selection import GridSearchCV

In [65]:
param_grid={
   
    'learning_rate':[0.05,0.1,0.15],
    'max_depth': [4,5,6],
    'n_estimators':[70,85,100]
    
}

In [66]:
grid= GridSearchCV(XGBClassifier(objective='binary:logistic'),param_grid, verbose=3)

In [67]:
grid.fit(train_x,train_y,verbose=True,early_stopping_rounds=10,eval_metric='aucpr',eval_set=[(test_x,test_y)])

c:\users\bhask\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\users\bhask\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV] learning_rate=0.05, max_depth=4, n_estimators=70 ................
[0]	validation_0-aucpr:0.52616
[1]	validation_0-aucpr:0.52616
[2]	validation_0-aucpr:0.52727
[3]	validation_0-aucpr:0.52713
[4]	validation_0-aucpr:0.53093
[5]	validation_0-aucpr:0.53094
[6]	validation_0-aucpr:0.54064
[7]	validation_0-aucpr:0.54550
[8]	validation_0-aucpr:0.54699
[9]	validation_0-aucpr:0.55176
[10]	validation_0-aucpr:0.55386
[11]	validation_0-aucpr:0.55394
[12]	validation_0-aucpr:0.55369
[13]	validation_0-aucpr:0.55428
[14]	validation_0-aucpr:0.55787
[15]	validation_0-aucpr:0.55758
[16]	validation_0-aucpr:0.55792
[17]	validation_0-aucpr:0.55799
[18]	validation_0-aucpr:0.55911
[19]	validation_0-aucpr:0.55930
[20]	validation_0-aucpr:0.56032
[21]	validation_0-aucpr:0.56121
[22]	validation_0-aucpr:0.56206
[23]	validation_0-aucpr:0.56212
[24]	validation_0-aucpr:0.56263
[25]	validation_0-aucpr:0.56320
[26]	validation_0-aucpr:0.56346
[27]	validatio

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.6s remaining:    0.0s


[0]	validation_0-aucpr:0.52089
[1]	validation_0-aucpr:0.52074
[2]	validation_0-aucpr:0.52933
[3]	validation_0-aucpr:0.52938
[4]	validation_0-aucpr:0.53127
[5]	validation_0-aucpr:0.53144
[6]	validation_0-aucpr:0.53160
[7]	validation_0-aucpr:0.53156
[8]	validation_0-aucpr:0.53982
[9]	validation_0-aucpr:0.53994
[10]	validation_0-aucpr:0.54477
[11]	validation_0-aucpr:0.54685
[12]	validation_0-aucpr:0.55839
[13]	validation_0-aucpr:0.55788
[14]	validation_0-aucpr:0.55934
[15]	validation_0-aucpr:0.55945
[16]	validation_0-aucpr:0.56090
[17]	validation_0-aucpr:0.56098
[18]	validation_0-aucpr:0.56248
[19]	validation_0-aucpr:0.56297
[20]	validation_0-aucpr:0.56291
[21]	validation_0-aucpr:0.56292
[22]	validation_0-aucpr:0.56332
[23]	validation_0-aucpr:0.56349
[24]	validation_0-aucpr:0.56422
[25]	validation_0-aucpr:0.56397
[26]	validation_0-aucpr:0.56543
[27]	validation_0-aucpr:0.56580
[28]	validation_0-aucpr:0.56664
[29]	validation_0-aucpr:0.56750
[30]	validation_0-aucpr:0.56743
[31]	validation_0-

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   15.3s remaining:    0.0s


[0]	validation_0-aucpr:0.52569
[1]	validation_0-aucpr:0.52569
[2]	validation_0-aucpr:0.53103
[3]	validation_0-aucpr:0.52986
[4]	validation_0-aucpr:0.53127
[5]	validation_0-aucpr:0.54209
[6]	validation_0-aucpr:0.54305
[7]	validation_0-aucpr:0.54583
[8]	validation_0-aucpr:0.55049
[9]	validation_0-aucpr:0.54921
[10]	validation_0-aucpr:0.54851
[11]	validation_0-aucpr:0.55532
[12]	validation_0-aucpr:0.55400
[13]	validation_0-aucpr:0.55710
[14]	validation_0-aucpr:0.55733
[15]	validation_0-aucpr:0.55720
[16]	validation_0-aucpr:0.55875
[17]	validation_0-aucpr:0.55943
[18]	validation_0-aucpr:0.55963
[19]	validation_0-aucpr:0.55918
[20]	validation_0-aucpr:0.56029
[21]	validation_0-aucpr:0.56128
[22]	validation_0-aucpr:0.56179
[23]	validation_0-aucpr:0.56157
[24]	validation_0-aucpr:0.56330
[25]	validation_0-aucpr:0.56386
[26]	validation_0-aucpr:0.56459
[27]	validation_0-aucpr:0.56609
[28]	validation_0-aucpr:0.56621
[29]	validation_0-aucpr:0.56620
[30]	validation_0-aucpr:0.56658
[31]	validation_0-

[Parallel(n_jobs=1)]: Done  81 out of  81 | elapsed: 16.7min finished


[0]	validation_0-aucpr:0.56442
[1]	validation_0-aucpr:0.56695
[2]	validation_0-aucpr:0.56752
[3]	validation_0-aucpr:0.56879
[4]	validation_0-aucpr:0.57149
[5]	validation_0-aucpr:0.57330
[6]	validation_0-aucpr:0.57422
[7]	validation_0-aucpr:0.57471
[8]	validation_0-aucpr:0.57604
[9]	validation_0-aucpr:0.57606
[10]	validation_0-aucpr:0.57613
[11]	validation_0-aucpr:0.57780
[12]	validation_0-aucpr:0.57839
[13]	validation_0-aucpr:0.57959
[14]	validation_0-aucpr:0.57958
[15]	validation_0-aucpr:0.58037
[16]	validation_0-aucpr:0.58058
[17]	validation_0-aucpr:0.58149
[18]	validation_0-aucpr:0.58200
[19]	validation_0-aucpr:0.58311
[20]	validation_0-aucpr:0.58371
[21]	validation_0-aucpr:0.58378
[22]	validation_0-aucpr:0.58448
[23]	validation_0-aucpr:0.58466
[24]	validation_0-aucpr:0.58482
[25]	validation_0-aucpr:0.58522
[26]	validation_0-aucpr:0.58550
[27]	validation_0-aucpr:0.58557
[28]	validation_0-aucpr:0.58578
[29]	validation_0-aucpr:0.58605
[30]	validation_0-aucpr:0.58613
[31]	validation_0-

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
       colsample_bynode=None, colsample_bytree=None, gamma=None,
       gpu_id=None, importance_type='gain', interaction_constraints=None,
       learning_rate=None, max_delta_step=None, max_depth=None,
       min_child_w...e,
       tree_method=None, use_label_encoder=True, validate_parameters=None,
       verbosity=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'learning_rate': [0.05, 0.1, 0.15], 'max_depth': [4, 5, 6], 'n_estimators': [70, 85, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [71]:
grid.best_params_

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 85}

In [70]:
new_model=XGBClassifier(learning_rate= 0.1, max_depth= 6, n_estimators= 85)
new_model.fit(train_x, train_y,verbose=True,early_stopping_rounds=10,eval_metric='aucpr',eval_set=[(test_x,test_y)])

[0]	validation_0-aucpr:0.56442
[1]	validation_0-aucpr:0.56695
[2]	validation_0-aucpr:0.56752
[3]	validation_0-aucpr:0.56879
[4]	validation_0-aucpr:0.57149
[5]	validation_0-aucpr:0.57330
[6]	validation_0-aucpr:0.57422
[7]	validation_0-aucpr:0.57471
[8]	validation_0-aucpr:0.57604
[9]	validation_0-aucpr:0.57606
[10]	validation_0-aucpr:0.57613
[11]	validation_0-aucpr:0.57780
[12]	validation_0-aucpr:0.57839
[13]	validation_0-aucpr:0.57959
[14]	validation_0-aucpr:0.57958
[15]	validation_0-aucpr:0.58037
[16]	validation_0-aucpr:0.58058
[17]	validation_0-aucpr:0.58149
[18]	validation_0-aucpr:0.58200
[19]	validation_0-aucpr:0.58311
[20]	validation_0-aucpr:0.58371
[21]	validation_0-aucpr:0.58378
[22]	validation_0-aucpr:0.58448
[23]	validation_0-aucpr:0.58466
[24]	validation_0-aucpr:0.58482
[25]	validation_0-aucpr:0.58522
[26]	validation_0-aucpr:0.58550
[27]	validation_0-aucpr:0.58557
[28]	validation_0-aucpr:0.58578
[29]	validation_0-aucpr:0.58605
[30]	validation_0-aucpr:0.58613
[31]	validation_0-

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=85, n_jobs=4, num_parallel_tree=1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       use_label_encoder=True, validate_parameters=1, verbosity=None)

In [72]:
from sklearn.metrics import accuracy_score
y_pred = new_model.predict(train_x)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(train_y,predictions)
accuracy

0.8039033295156592

In [73]:
y_pred = new_model.predict(test_x)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(test_y,predictions)
accuracy

0.7989771833202203

In [74]:
import pickle
filename = 'xgboost_model2.pickle'
pickle.dump(new_model, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))

In [164]:
dft=pd.read_csv(r'C:\Users\bhask\OneDrive\Documents\KN-ML\Ineuron Materials\test.csv')
df=pd.read_csv(r'C:\Users\bhask\OneDrive\Documents\KN-ML\Ineuron Materials\train.csv')


In [165]:
dft.sample(1)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active
12935,BZOTEFT3,Male,71,RG282,Other,X2,39,Yes,506365,No


In [166]:
dft_id=dft['ID']

In [169]:
dft.isnull().sum()

ID                         0
Gender                     0
Age                        0
Region_Code                0
Occupation                 0
Channel_Code               0
Vintage                    0
Credit_Product         12522
Avg_Account_Balance        0
Is_Active                  0
dtype: int64

In [170]:
def impute_nan(df,i):
    df[i+"_random"]=df[i]
    random_sample=df[i].dropna().sample(df[i].isnull().sum(),random_state=0)
    random_sample.index=df[df[i].isnull()].index
    df.loc[df[i].isnull(),i+'_random']=random_sample
    df.drop([i],axis=1,inplace=True)

In [171]:
impute_nan(dft,'Credit_Product')

In [172]:
dft.isnull().sum()

ID                       0
Gender                   0
Age                      0
Region_Code              0
Occupation               0
Channel_Code             0
Vintage                  0
Avg_Account_Balance      0
Is_Active                0
Credit_Product_random    0
dtype: int64

In [173]:
dft_age=dft[['Gender','Credit_Product_random','Is_Active']]

In [174]:
dft_age = pd.get_dummies(dft_age,drop_first=True)

In [175]:
dft.drop(['Gender'],axis=1,inplace=True)
dft.drop(['Credit_Product_random'],axis=1,inplace=True)
dft.drop(['Is_Active'],axis=1,inplace=True)

In [176]:
dft=pd.concat([dft,dft_age],axis=1)

In [177]:
dft.size

1053120

In [178]:
def ordinal1(df,i,j):
    ordinal_labels=df.groupby([i])[j].mean().sort_values().index
    enumerate(ordinal_labels,0)
    return ordinal_labels

In [179]:
def ordinal2(df,i,j):
    
    ordinal_labels2={k:i for i,k in enumerate(j,0)}
    ordinal_labels2
    df[i+'_new']=df[i].map(ordinal_labels2)
    df.drop(i,inplace=True,axis=1)

In [180]:
j=ordinal1(df,'Region_Code','Is_Lead')

In [181]:
ordinal2(dft,'Region_Code',j)

In [182]:
j=ordinal1(df,'Occupation','Is_Lead')

In [183]:
ordinal2(dft,'Occupation',j)

In [184]:
j=ordinal1(df,'Channel_Code','Is_Lead')

In [185]:
ordinal2(dft,'Channel_Code',j)

In [186]:
dft=dft.drop('ID',axis=1)

In [187]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_data=scaler.fit_transform(dft)

c:\users\bhask\appdata\local\programs\python\python37\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
c:\users\bhask\appdata\local\programs\python\python37\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [188]:
loaded_model = pickle.load(open('xgboost_model2.pickle', 'rb'))

In [189]:
pred=loaded_model.predict(scaled_data)

In [190]:
df_pred = pd.DataFrame(pred, columns = ['Is_Lead'])

In [191]:
df_pred.head()

,Is_Lead
0,0
1,0
2,0
3,0
4,0


In [192]:
df5=pd.concat([dft_id,df_pred],ignore_index=True,axis=1)

In [193]:
df5.to_csv(r'C:\Users\bhask\OneDrive\Documents\KN-ML\Ineuron Materials\pred_b.csv', index = False)